In [1]:
import numpy as np
np.random.seed(12)

def get_bootstrap(data, labels, N):
    '''
    функция формирует выборки
    :param data: датасет, labels: признаки, N: кол-во деревье в ансамбле
    :return bootstrap: кортеж с выборкой и OOB-признаки
    '''
    n_samples = data.shape[0]
    bootstrap = []
    
    for i in range(N):
        b_data = np.zeros(data.shape)
        b_labels = np.zeros(labels.shape)
        s_set = set()
        
        for j in range(n_samples):
            sample_index = np.random.randint(0, n_samples-1)
            b_data[j] = data[sample_index]
            b_labels[j] = labels[sample_index]
            s_set.add(sample_index)
            
        out_of_bag_data = np.delete(data, list(s_set), axis=0)
        out_of_bag_labels = np.delete(labels, list(s_set), axis=0)
        
        bootstrap.append((b_data, b_labels, out_of_bag_data, out_of_bag_labels))

    return bootstrap

def get_subsample(len_sample):
    '''
    функция реализует выбор индексов случайных признаков
    :param len_sample: общее количество признаков
    :return subsample: sqrt из перемешанных индексов len_sample
    '''
    sample_indexes = [i for i in range(len_sample)]
    len_subsample = int(np.sqrt(len_sample))
    np.random.shuffle(sample_indexes)
    subsample = [sample_indexes.pop() for _ in range(len_subsample)]
        
    return subsample

# класс узла
class Node:
    count = 0

    def __init__(self, index, t, true_branch, false_branch, level):
        self.index = index                 # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t                         # значение порога
        self.true_branch = true_branch     # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch   # поддерево, не удовлетворяющее условию в узле
        self.level = level
        Node.count += 1
        
# класс листа (терминального узла)
class Leaf:
    count = 0

    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        Leaf.count += 1

    def predict(self):
        '''возвращает класс, количество объектов которого максимально в конкретном'''
        classes = {}
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        prediction = max(classes, key=classes.get)
        return prediction

# класс-счётчик для ограничения количества узлов
class NodesCount:
    count = 0

    def __init__(self, max_nodes):
        self.max_nodes = max_nodes
        NodesCount.count += 1

    def max_node(self):
        if self.max_nodes == 0:
            self.max_nodes = 1e15
        if NodesCount.count-1 >= self.max_nodes:
            return 'stop'


# класс-счётчик для ограничения количества листов
class LeafsCount:
    count = 0

    def __init__(self, max_leafs):
        self.max_leafs = max_leafs
        LeafsCount.count += 1

    def max_leaf(self):
        if self.max_leafs == 0:
            self.max_leafs = 1e15
        if LeafsCount.count >= self.max_leafs:
            return 'stop'
        
def reset_counters():
    '''сбросить счёткики классов NodesCount и LeafsCount'''
    Node.count, Leaf.count, NodesCount.count, LeafsCount.count = 0, 0, 0, 0
    
def gini(labels):
    '''
    индекс Джини
    :return impurity: коэффициент неопределенности Джини
    '''
    classes, impurity = {}, 1
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
        
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

def quality(left_labels, right_labels, current_gini):
    '''
    Расчет качества Q
    :return Q: качество разбиения
    '''
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

def split(data, labels, index, t):
    '''
    Разбиение датасета в узле
    :return разбитые данные true_data, false_data, true_labels, false_labels
    '''
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data, false_data = data[left], data[right]
    true_labels, false_labels = labels[left], labels[right]
        
    return true_data, false_data, true_labels, false_labels

def find_best_split(data, labels, min_leaf):
    '''
    Функция находит наилучшее разбиение
    :param data: данные, labels: признаки, min_leaf: минимальное кол-во объектов в узле
    :return best_quality, best_t, best_index
    '''
    current_gini = gini(labels)
    best_quality, best_t, best_index = 0, None, None
    n_features = data.shape[1]
    
    # выбор индекса из подвыборки длиной sqrt(n_features)
    subsample = get_subsample(n_features)
    
    for index in subsample:
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее min_leaf объекта
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index


def build_tree(data, labels, min_leaf, depth, count_nodes, count_leafs, level=0):
    '''
    Рекурсивное построение дерева
    :param data: данные, labels: признаки, min_leaf: минимальное кол-во объектов в узле
           depth: максимально допустимое количество уровней дерева, 
           count_nodes: максимально допустимое количество узлов 
           count_leafs: максимально допустимое количество узлов, level=0: счётчик
    :return Leaf or Node: экземпляр класса Leaf или Node
    '''

    quality, t, index = find_best_split(data, labels, min_leaf)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)
    
    # прекращаем рекурсию, когда достигнуто максимальное количество уровней
    try:
        if depth and level >= depth:
            return Leaf(data, labels=labels)
    except:
        pass

    # прекращаем рекурсию, когда достигнуто максимальное количество узлов
    step_nodes = NodesCount(count_nodes)
    if step_nodes.max_node() == 'stop':
        return Leaf(data, labels=labels)
    # прекращаем рекурсию, когда достигнуто максимальное количество листьев
    step_leafs = LeafsCount(count_leafs)
    if step_leafs.max_leaf() == 'stop':
        return Leaf(data, labels=labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    level += 1
    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, min_leaf, depth, count_nodes, count_leafs, level)
    false_branch = build_tree(false_data, false_labels, min_leaf, depth, count_nodes, count_leafs, level)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch, level)

def random_forest(data, labels, n_trees, min_leaf=1, depth=0, count_nodes=0, count_leafs=0):
    '''
    функция формирования случайного леса
    :param data: данные, labels: признаки, min_leaf: минимальное кол-во объектов в узле
           depth: максимально допустимое количество уровней дерева, 
           count_nodes: максимально допустимое количество узлов 
           count_leafs: максимально допустимое количество узлов
    :return forest: лес, accuracy: значение accuracy на OOB-признаках
    '''
    forest, accuracy_list = [], []
    bootstrap = get_bootstrap(data, labels, n_trees)
    
    for b_data, b_labels, out_of_bag_data, out_of_bag_labels in bootstrap:
        tree = build_tree(b_data, b_labels, min_leaf, depth, count_nodes, count_leafs)
        forest.append(tree)
        reset_counters()
        
        # считаем accuracy для OOB-признаков каждого дерева
        accuracy_list.append(accuracy_metric(out_of_bag_labels, tree_vote([tree], out_of_bag_data)))
    accuracy = sum(accuracy_list)/len(accuracy_list)
    
    return forest, accuracy

def classify_object(obj, node):
    '''
    функция определения класса объекта
    :return answer: предсказание класса
    '''

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf): 
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)
    

def predict(data, tree):
    '''
    функция формирования предсказания по выборке на одном дереве
    '''
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
        
    return classes

def tree_vote(forest, data):
    '''
    функция сформирует список с предсказаниями для каждого объекта
    возвращает предсказание голосованием деревьев и 
    возвращает в качестве итогового предсказания для каждого объекта то,
    за которое проголосовало большинство деревьев
    '''
    predictions = [predict(data, tree) for tree in forest]
    predictions_per_object = list(zip(*predictions))
    voted_predictions = [max(set(obj), key=obj.count) for obj in predictions_per_object]
       
    return voted_predictions

def accuracy_metric(actual, predicted):
    '''
    функция для подсчёта метрики accuracy
    :param actual: истинные значения, predicted: предсказанные значения
    :return accuracy
    '''
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0